In [1]:
import Pkg
Pkg.activate("../../../FinancialPlanner")

  Activating project at `c:\Users\matsz\programowanie\Optymalizacja_portfela\FinancialPlanner`


In [2]:
using Revise
using LinearAlgebra
using Distributions
using PDMats
using StatsBase
using DataFrames, TimeSeries, XLSX
using PrettyTables

┌ Warning: attempting to remove probably stale pidfile
│   path = C:\Users\matsz\.julia\compiled\v1.11\DataFrames\AR9oZ_d1gGA.ji.pidfile
└ @ FileWatching.Pidfile C:\Users\matsz\.julia\juliaup\julia-1.11.2+0.x64.w64.mingw32\share\julia\stdlib\v1.11\FileWatching\src\pidfile.jl:249


In [4]:
include("../VARs/utils.jl")
includet("../bootstrap_model.jl")

In [5]:
df = DataFrame(XLSX.readtable("../../data/usa_data.xlsx", "Data", infer_eltypes=true))
data_source = collapse(TimeArray(df; timestamp = :Date), Dates.month, last)

freq = 12
returns = percentchange(data_source[:Total_Stock, :Price_10Y], :log)
cpi = percentchange(data_source[:CPI], :log)
real_returns = returns .- cpi
start = Date(1955, 01,01)
end_d = Date(2024, 12, 31)


display("Returns")
display(returns_summarystats(to(from(returns, start),end_d), freq))
display("Real_returns")
display(returns_summarystats(to(from(real_returns, start),end_d), freq))


display("Correlations")
display(cor(values(to(from(real_returns, start),end_d)))) 

"Returns"

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock,0.1473,1.8543,0.1012,0.4365,-2.9008,-0.6469,-0.1883,0.0258,2.1941,0.6874,0.1474
Price_10Y,0.0755,1.3969,0.0531,0.1909,-0.9866,0.3097,-0.0893,0.1022,1.9867,0.7037,0.0377


nothing

"Real_returns"

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock_CPI,0.1479,1.7607,0.0658,0.4053,-2.9112,-0.596,-0.2186,0.032,1.9119,0.445,0.1107
Price_10Y_CPI,0.078,1.279,0.0177,0.1544,-1.1677,0.2268,-0.1322,0.1437,2.0153,0.2268,0.0058


nothing

"Correlations"

2×2 Matrix{Float64}:
 1.0        0.0980477
 0.0980477  1.0

In [6]:
assets_names = [:Total_Stock_CPI, :Price_10Y_CPI]
data = to(from(real_returns[assets_names], start),end_d) 

scenarios = block_bootstrap(values(data), 10_000, block_size=60, random_block=true)

periods = [1,5, 10, 25]
ret_in_years = cum_returns_in_periods(scenarios, periods, freq, true)
print_scenarios_summary(ret_in_years, assets_names, string.(periods))
n_assets = length(assets_names)
for a in 1:n_assets
    print_scenarios_percentiles(ret_in_years[a, :, :], [.01, .025, .05, .25, .5, .75, .95, .975, .99], string.(periods), string.(assets_names[a]))
end  


,Total_Stock_CPI,Price_10Y_CPI
1,0.0635,0.0247
5,0.0632,0.0235
10,0.0619,0.0223
25,0.0623,0.0222


,Total_Stock_CPI,Price_10Y_CPI
1,0.1567,0.0865
5,0.0681,0.0415
10,0.0483,0.0311
25,0.0301,0.0204


,Total_Stock_CPI,Price_10Y_CPI
1,-0.7773,0.5244
5,-0.2652,0.2761
10,-0.2955,0.1245
25,-0.1677,0.0801


,Total_Stock_CPI,Price_10Y_CPI
1,1.1654,1.0231
5,-0.1454,0.6981
10,-0.2771,-0.0371
25,-0.0992,-0.1397


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.4252,-0.2905,-0.2157,-0.0182,0.0818,0.1658,0.286,0.3119,0.3653
5,-0.0959,-0.0737,-0.0514,0.0149,0.0725,0.1099,0.1696,0.1927,0.2104
10,-0.0514,-0.0362,-0.0229,0.0295,0.0656,0.0975,0.1346,0.1458,0.1604
25,-0.0101,0.0017,0.0118,0.0416,0.0634,0.0836,0.1098,0.1173,0.1277


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.1572,-0.1238,-0.0996,-0.0341,0.0196,0.0785,0.1725,0.2263,0.285
5,-0.0737,-0.0578,-0.0414,-0.0035,0.0219,0.0496,0.0928,0.1146,0.1419
10,-0.0468,-0.037,-0.0271,0.0007,0.022,0.0427,0.0753,0.0865,0.0963
25,-0.0223,-0.0166,-0.0113,0.0083,0.0218,0.036,0.0564,0.0629,0.0703


In [7]:
mdd, mddl = max_drawdown_and_length(scenarios[1,:,:])

display(assets_names[1])
display("mdd")
display(quantile(mdd, [.03, .25, .5, .75, .97])')
display("mddl")
display(quantile(mddl, [.03, .25, .5, .75, .97])')

mdd, mddl = max_drawdown_and_length(scenarios[2,:,:])

display(assets_names[2])
display("mdd")
display(quantile(mdd, [.03, .25, .5, .75, .97])')
display("mddl")
display(quantile(mddl, [.03, .25, .5, .75, .97])')

:Total_Stock_CPI

"mdd"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 0.406639  0.521378  0.539183  0.592069  0.720677

"mddl"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 60.0  107.0  145.0  190.0  351.03

:Price_10Y_CPI

"mdd"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 0.19893  0.376498  0.431628  0.496197  0.650372

"mddl"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 79.0  156.0  223.0  335.0  658.0

In [29]:
rr = values(to(from(real_returns[assets_names], start),end_d) ) .- mean(values(to(from(real_returns[assets_names], start),end_d) ), dims=1) .+ [.042 .025] ./ 12


scenarios = block_bootstrap(rr, 10_000, block_size=60, random_block=true)

periods = [1,5, 10, 25]
ret_in_years = cum_returns_in_periods(scenarios, periods, freq, true)
print_scenarios_summary(ret_in_years, assets_names, string.(periods))
n_assets = length(assets_names)
for a in 1:n_assets
    print_scenarios_percentiles(ret_in_years[a, :, :], [.01, .025, .05, .25, .5, .75, .95, .975, .99], string.(periods), string.(assets_names[a]))
end  


,Total_Stock_CPI,Price_10Y_CPI
1,0.0409,0.0321
5,0.0388,0.031
10,0.0388,0.0299
25,0.0386,0.0297


,Total_Stock_CPI,Price_10Y_CPI
1,0.1534,0.0867
5,0.0682,0.0414
10,0.0481,0.0308
25,0.0298,0.0201


,Total_Stock_CPI,Price_10Y_CPI
1,-0.7535,0.542
5,-0.273,0.2852
10,-0.309,0.1307
25,-0.1707,0.0926


,Total_Stock_CPI,Price_10Y_CPI
1,1.0722,1.1241
5,-0.2277,0.7147
10,-0.3105,-0.0611
25,-0.1938,-0.1963


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.4119,-0.3107,-0.2392,-0.0436,0.0613,0.1425,0.2618,0.2876,0.3377
5,-0.118,-0.0986,-0.0771,-0.0105,0.0476,0.0863,0.1434,0.165,0.1841
10,-0.0753,-0.0593,-0.0475,0.0062,0.0429,0.0742,0.1107,0.1219,0.1349
25,-0.0329,-0.0226,-0.0118,0.0185,0.0394,0.0598,0.0853,0.0933,0.1022


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.1561,-0.1152,-0.0923,-0.0261,0.027,0.0852,0.1819,0.234,0.2952
5,-0.0671,-0.051,-0.0337,0.0041,0.0292,0.0572,0.0999,0.1208,0.1483
10,-0.0393,-0.0275,-0.0186,0.008,0.0296,0.05,0.0824,0.0928,0.1032
25,-0.0147,-0.0085,-0.0029,0.0156,0.0293,0.0435,0.0634,0.07,0.0768


In [31]:
mdd, mddl = max_drawdown_and_length(scenarios[1,:,:])

display(assets_names[1])
display("mdd")
display(quantile(mdd, [.03, .25, .5, .75, .97])')
display("mddl")
display(quantile(mddl, [.03, .25, .5, .75, .97])')

mdd, mddl = max_drawdown_and_length(scenarios[2,:,:])

display(assets_names[2])
display("mdd")
display(quantile(mdd, [.03, .25, .5, .75, .97])')
display("mddl")
display(quantile(mddl, [.03, .25, .5, .75, .97])')

:Total_Stock_CPI

"mdd"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 0.456528  0.549323  0.616923  0.673091  0.816195

"mddl"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 76.0  158.0  212.0  303.0  580.03

:Price_10Y_CPI

"mdd"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 0.181928  0.360337  0.396304  0.440609  0.592464

"mddl"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 60.0  121.0  171.0  254.0  512.03